In [1]:
%matplotlib inline 

# Caching Rest Data Investigation
***

## Table of Contents
***
* [Aim](#aim)
* [Code Setup](#code-setup)
* [Ehcache Data](#ehcache)
* [Hazelcast Data](#hazelcast)
* [Memcached Data](#memcached)
* [Redis Data](#redis)
* [Cache Size Comparison](#cache_size_comparison)
* [Eviction Policy Comparison](#eviction_policy_comparison)

## Aim <a class="anchor" id="aim"></a>
***

The aim of this notebook is to do some Exploratory data analysis on the data obtained from the Caching Rest Application

## Code Setup <a class="anchor" id="code-setup"></a>
***

### Import Libraries

In [2]:
import pymysql
import pandas as pd
import sys
import re
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")
plt.rcParams['figure.figsize'] = 14, 8
IMAGE_POOL_SIZE_MB = 2048
DECIMAL_ROUND_SIZE = 6

### Custom Classes

In [3]:
class CachePerformanceMetric:
    def __init__(self, cache_hit_size, cache_miss_size):
        self.cache_hit_size = cache_hit_size
        self.cache_miss_size = cache_miss_size
        self.total_size = cache_hit_size + cache_miss_size
        self.hit_ratio = cache_hit_size/self.total_size
    
    def __repr__(self):
        return "Hit Ratio : " + str(self.hit_ratio)

class MainCacheAnalyseData:
    def __init__(self, batch_ids, cache_performance_metrics, cache_initial_content_per_batch,
                cache_performance_per_batch, cache_remainder_per_batch, agg_df):
        self.batch_ids = batch_ids
        self.cache_performance_metrics = cache_performance_metrics
        self.cache_initial_content_per_batch = cache_initial_content_per_batch
        self.cache_performance_per_batch = cache_performance_per_batch
        self.cache_remainder_per_batch = cache_remainder_per_batch
        self.agg_df = agg_df
    def __iter__(self):
        yield 'batch_ids', self.batch_ids
        yield 'cache_performance_metrics', self.cache_performance_metrics
        yield 'cache_initial_content_per_batch', self.cache_initial_content_per_batch
        yield 'cache_performance_per_batch', self.cache_performance_per_batch
        yield 'cache_remainder_per_batch', self.cache_remainder_per_batch
        yield 'agg_df', self.agg_df

### Constants

In [4]:
db_connection = pymysql.connect("pidb","dbuser","password","cachetest" )

### Useful Functions

#### Table Read

In [5]:
def read_full_db_table(table_name):
    return pd.read_sql('SELECT * FROM ' + table_name, con=db_connection)

#### Query to Dataframe

In [6]:
def obtain_df_from_query(query_string):
    return pd.read_sql(query_string, con=db_connection)

#### Cache Hit Bytes to Boolean

In [7]:
def convert_cache_hit_to_boolean(df):
    df.cache_hit = df.cache_hit.map({b'\x00' : False, b'\x01' : True})
    return df

#### DF ids to comma seperated string

In [8]:
def df_ids_to_string(df):
    id_list = df.id.tolist()
    id_list = [str(i) for i in id_list]
    return ','.join(id_list)

#### Compute Cache Size from Dataframe

In [9]:
def cache_size_mb_from_cache_content(df):
    return (df[["file_sizekb"]].sum().values[0])/1024

#### Obtain Colours for DF

In [10]:
def obtain_colours_for_length(length):
    cmap = plt.get_cmap('Set3')
    # https://matplotlib.org/api/pyplot_summary.html
    indices = np.linspace(0, cmap.N, length)
    my_colors = [cmap(int(i)) for i in indices]
    return my_colors

#### Obtain Label

In [11]:
def obtain_label(label, field):
    temp_label = label
    if "time" in field:
        field_items = field.split("_")
        number_of_filed_items = len(field_items)
        time_unit = field_items[number_of_filed_items-1]
        temp_label = temp_label + ' (' + time_unit + ')'
    return temp_label

#### DF to Latex Table

In [12]:
# I know there is a to_latex method but it wasn't outputting how I wanted it
def write_df_to_latex_table(df, file_name, table_caption):
    table_label = table_caption.replace(" ", "_")
    df_without_batch = df.copy()
    df_without_batch = df_without_batch.drop('batch_id', axis=1)
    means = np.around(df_without_batch.mean(), decimals=DECIMAL_ROUND_SIZE)
    medians = np.around(df_without_batch.median(), decimals=DECIMAL_ROUND_SIZE)
    modes =  np.around(df_without_batch.mode().iloc[0], decimals=DECIMAL_ROUND_SIZE)
    with open(output_dir + file_name, "w") as f:
        # Table Start
        f.write("\\begin{table}[H]\n")
        f.write("\\begin{center}\n")
        f.write("\\begin{adjustbox}{width=1\\textwidth}\n")
        f.write("\\begin{tabular}{|" + "".join(["c"] * len(df.columns)) + "|}\n")
        
        # Column Names
        f.write("\\hline \n")
        f.write(" & ".join(['\\textbf{' + str(x).replace("_", " ") + '}' for x in df.columns.values]) + " \\\\ [0.5ex] \n")
        f.write("\\hline \\hline \n")
        
        # The Batch Data
        for i, row in df.iterrows():
            f.write(" & ".join([re.sub('\.0$', '',str(x)) for x in row.values]) + " \\\\\n")
            f.write("\\hline \n")
        
        # Extra Spacing
        f.write("\\hline \\hline \n")
        
        # re.sub('\.0$', '',str(x) replaces anything that end in .0 with nothing, mainly wanted to get rid
        # of .0 with ids
        
        # Mean
        f.write("\\textbf{Mean} & " + " & ".join([re.sub('\.0$', '',str(x)) for x in means]) + " \\\\\n")
        f.write("\\hline \n")
        
        # Median
        f.write("\\textbf{Median} & " + " & ".join([re.sub('\.0$', '',str(x)) for x in medians]) + " \\\\\n")
        f.write("\\hline \n")

        # Mode
        f.write("\\textbf{Mode} & " + " & ".join([re.sub('\.0$', '',str(x)) for x in modes]) + " \\\\\n")
        f.write("\\hline \n")
        
        # End Table
        f.write("\\end{tabular} \n")
        f.write("\\end{adjustbox} \n")
        f.write("\\caption{" + table_caption + "} \n")
        f.write("\\label{table:" + table_label + "} \n")
        f.write("\\end{center} \n")
        f.write("\\end{table}")

#### Process a Set of Parameters

In [13]:
def crunch_the_data(cache_type, cache_size_mb, eviction_policy):
    query_for_batch = ("cache_type == '" + cache_type + "' and cache_size_mb == '" + str(cache_size_mb) 
                       + "' and eviction_policy == '" + eviction_policy + "'")
    # Batches fitting criteria
    batches_df = batch_df.query(query_for_batch)
    
    if(batches_df.empty):
        print("Sorry, No batches for :", cache_type, ",", cache_size_mb, ",",  eviction_policy)
        return None
    
    # Cache Performance
    cp_query = ("SELECT * FROM cache_performance where batch_id in ("
                                              + df_ids_to_string(batches_df)
                                              + ")")
    cache_performace_df = obtain_df_from_query(cp_query)
    # Issue with Booleans coming in as bytes and not booleans
    cache_performace_df = convert_cache_hit_to_boolean(cache_performace_df)
    
    # Cache Initial Content
    cache_initial_content_df = obtain_df_from_query("SELECT * FROM cache_initial_content where batch_id in ("
                                              + df_ids_to_string(batches_df)
                                              + ")")
    # Cache Remainder
    cache_remainder_df = obtain_df_from_query("SELECT * FROM cache_remainder where batch_id in ("
                                              + df_ids_to_string(batches_df)
                                              + ")")

    # Separate data into dictionary based on batches
    cache_performance_per_batch = {}
    cache_initial_content_per_batch = {}
    cache_remainder_per_batch = {}
    batch_ids = batches_df.id.unique().tolist()
    for batch_id in batch_ids :
        query_for_df = "batch_id == " + str(batch_id)
        cache_performance_per_batch[batch_id] = cache_performace_df.query(query_for_df)
        cache_initial_content_per_batch[batch_id] = cache_initial_content_df.query(query_for_df)
        cache_remainder_per_batch[batch_id] = cache_remainder_df.query(query_for_df)
    
    # Compute the Cache Performance Metrics
    cache_performance_metrics = []
    cache_size_mb_initial_list = []
    element_count_at_start_list = []
    element_count_testing_list = []
    number_of_hits_list = []
    number_of_misses_list = []
    hit_ratio_list = []
    cache_size_mb_end_list = []
    element_count_at_end_list = []
    elapsed_time_ns_list = []
    avg_response_time_list = []
    
    for batch_id in batch_ids :
        cp_df = cache_performance_per_batch[batch_id]
        cic_df = cache_initial_content_per_batch[batch_id]
        cr_df = cache_remainder_per_batch[batch_id]
        
        # Cache MB in Cache at Start
        initial_full_cache_size_mb = cache_size_mb_from_cache_content(cic_df)
        element_count_at_start = cic_df.shape[0]
        
        element_count_testing = cp_df.shape[0]
        # Calculate Hit Ratio
        cp_df_hit = cp_df.query("cache_hit")
        cp_df_miss = cp_df.query("cache_hit == False")
        hit_size = cp_df_hit.shape[0]
        miss_size = cp_df_miss.shape[0]
        cache_performance_metric = CachePerformanceMetric(hit_size, miss_size)
        cache_performance_metrics.append(cache_performance_metric)
        
        # Cache MB in Cache at End
        end_full_cache_size_mb = cache_size_mb_from_cache_content(cr_df)
        element_count_at_end = cr_df.shape[0]
        
        # Elapsed Time Nano Seconds - time for all the testing (only sending of the data this excludes cache fill and checking what is in cache)
        etns = cp_df.elapsed_time_inns.sum()
        
        avg_response_time = cp_df.elapsed_time_inns.mean()
        
        # Add Aggregate Data to lists
        cache_size_mb_initial_list.append(initial_full_cache_size_mb)
        element_count_at_start_list.append(element_count_at_start)
        element_count_testing_list.append(element_count_testing)
        number_of_hits_list.append(hit_size)
        number_of_misses_list.append(miss_size)
        hit_ratio_list.append(cache_performance_metric.hit_ratio)
        cache_size_mb_end_list.append(end_full_cache_size_mb)
        element_count_at_end_list.append(element_count_at_end)
        elapsed_time_ns_list.append(etns)
        avg_response_time_list.append(avg_response_time)
   
    # Gen Aggregate Data DF
    agg_data = {"batch_id": batch_ids, 
                "cache_size_mb_start" : cache_size_mb_initial_list,
                "element_count_start" : element_count_at_start_list,
                "element_count_sent" : element_count_testing_list,
                "elapsed_time_ns": elapsed_time_ns_list,
                "avg_response_time_ns" : avg_response_time_list,
                "hits" : number_of_hits_list,
                "misses" : number_of_misses_list,
                "hit_ratio" : hit_ratio_list,
                "cache_size_mb_end" : cache_size_mb_end_list,
                "element_count_end" : element_count_at_end_list
                }
    agg_df = pd.DataFrame(data=agg_data)
     
    # Average Hit Ratio
    average_hit_ratio = agg_df.hit_ratio.mean()
    print("Average Hit Ratio :", average_hit_ratio)
    
    # Generate the important data made from this incase it is needed elsewhere
    main_analyse_data = MainCacheAnalyseData(batch_ids, cache_performance_metrics, cache_initial_content_per_batch,
                                            cache_performance_per_batch, cache_remainder_per_batch, agg_df)
    
    # Export the Agg Data
    agg_file_name = cache_type + "_" + str(cache_size_mb) + "MB_" + eviction_policy
    table_caption = cache_type + " " + str(cache_size_mb) + "MB " + eviction_policy
    
    # Round the data to 6 decimal places
    agg_df = agg_df.round(DECIMAL_ROUND_SIZE)
    # Too many columns so need to split
    write_df_to_latex_table(agg_df[["batch_id", 
                                    "cache_size_mb_start", 
                                    "cache_size_mb_end",
                                    "element_count_start",
                                    "element_count_end"]], agg_file_name + "_size.tex", table_caption + " Sizes")

    write_df_to_latex_table(agg_df[["batch_id", 
                                    "element_count_sent",
                                    "elapsed_time_ns",
                                    "hits",
                                    "misses",
                                    "hit_ratio"]], agg_file_name + "_ratio.tex", table_caption + " Hit Ratio")
    
    # Tried being smart but dictionary was just easiest way to go
    return dict(main_analyse_data)

#### Obtain Batch Info from Id

In [14]:
def obtain_batch_info(batch_id):
    batch_info = batch_df.query("id == " + str(batch_id))
    return batch_info

#### Plot Avg Hit Ratio

In [15]:
def plot_avg(graph_title, field, order_descending, *cache_analyse_data):
    if(len(cache_analyse_data) <= 0):
        print("Can't plot what I don't have")
        return
    cache_list = []
    avg_list = []
    standard_deviation_list = []
    cache_size_mb = 0
    # Going to assume that all batch ids of main data analyse point to same cache type
    for main_cache_analyse_data in  cache_analyse_data:
        batch_info = obtain_batch_info(main_cache_analyse_data["batch_ids"][0])
        stand_dev = main_cache_analyse_data["agg_df"][field].std()
        standard_deviation_list.append(stand_dev)
        avg = main_cache_analyse_data["agg_df"][field].mean()
        cache_type = batch_info.cache_type.tolist()[0]
        cache_size_mb = int(batch_info.cache_size_mb.tolist()[0])
        eviction_policy = batch_info.eviction_policy.tolist()[0]
        cache_list.append(cache_type + "-" + str(cache_size_mb) + "MB-" + eviction_policy)
        avg_list.append(avg)
    
    d = {"cache": cache_list, "avg" : avg_list, "standard_deviation" : standard_deviation_list}
    df = pd.DataFrame(data=d)
    if(order_descending):
        df = df.sort_values(by="avg", ascending=False)
    
    fig, ax = plt.subplots()

    my_colors = obtain_colours_for_length(df.shape[0])
    
    df['avg'].plot.bar(yerr=df['standard_deviation'], color=my_colors)
    
    ax.set_xticklabels(df["cache"], rotation='vertical')
    
    if(field == "hit_ratio"):
        expected_hit_ratio = cache_size_mb/IMAGE_POOL_SIZE_MB
        ax.axhline(expected_hit_ratio, color="gray")
        ax.text(1.02, expected_hit_ratio, "Expected Ratio : " + str(expected_hit_ratio), va='center', ha="left", bbox=dict(facecolor="w",alpha=0.5),
            transform=ax.get_yaxis_transform())
        
    plt.xlabel('Cache Type')
    y_label = 'Avg'
    y_label = obtain_label(y_label, field)
    
    plt.ylabel(y_label)
    plt.title(graph_title)
    
    file_name = graph_title.replace(" ", "_") + ".pdf"
    fig.savefig(output_dir + file_name, bbox_inches='tight')
    plt.show()

#### Plot Cache Size vs Hit Ratio For Multiple Caches

In [16]:
def plot_cache_size_vs_avg_line(title, field, *cache_analyse_data_list):
    if(len(cache_analyse_data_list) <= 0):
        print("Can't plot what I don't have")
        return
    
    legend=[]
    file_name = title.replace(" ", "_")
    fig, ax = plt.subplots()
    
    my_colors = obtain_colours_for_length(len(cache_analyse_data_list))
    
    for the_index, cache_analyse_data in enumerate(cache_analyse_data_list):
        cache_list = []
        eviction_policy_list = []
        cache_size_list = []
        avg_list = []
        # Going to assume that all batch ids of main data analyse point to same cache type and size
        # Also going to assume that all analyse data fed into this method belong to same cache type and eviction policy
        for main_cache_analyse_data in  cache_analyse_data:
            batch_info = obtain_batch_info(main_cache_analyse_data["batch_ids"][0])
            avg = main_cache_analyse_data["agg_df"][field].mean()
            cache_type = batch_info.cache_type.tolist()[0]
            cache_size = batch_info.cache_size_mb.tolist()[0]
            eviction_policy = batch_info.eviction_policy.tolist()[0]

            cache_list.append(cache_type)
            eviction_policy_list.append(eviction_policy)
            cache_size_list.append(int(cache_size))
            avg_list.append(avg)

        if(len(set(cache_list)) != 1) or (len(set(eviction_policy_list)) != 1):
            print("The cache type and eviction policy must be constant to use this function only cache size may change")
            return
        
        file_name = file_name + "_" + cache_list[0] + "_" + eviction_policy_list[0]
        d = {"cache_size": cache_size_list, "avg" : avg_list}
        df = pd.DataFrame(data=d)
        df = df.sort_values(by=["cache_size"], ascending=True)

        plt.plot(df["cache_size"], df["avg"], "-o", color=my_colors[the_index])
        legend.append(cache_list[0] + " - " + eviction_policy_list[0])
        
    plt.xlabel('Cache Size (MB)')
    
    y_label = 'Avg'
    y_label = obtain_label(y_label, field)
    
    plt.ylabel(y_label)
    plt.title(title)
    lg = plt.legend(legend, frameon=True)
    lg.get_frame().set_facecolor('white')
    plt.show()
    fig.savefig(output_dir + file_name + ".pdf", bbox_inches='tight')

#### Plot Hit Ratio Variation

In [17]:
def plot_hit_ratio_variation(title, cache_analyse_data, percentage_sizes):
    if(100 % percentage_sizes != 0):
        print(percentage_sizes, "does not divide into 100 evenly")
        return
    number_of_segements = 100/percentage_sizes
    cache_performance_per_batch = cache_analyse_data['cache_performance_per_batch']
    
    my_colors = obtain_colours_for_length(len(cache_performance_per_batch.keys()))
    
    file_name = title.replace(" ", "_")
    
    legend=[]
    fig, ax = plt.subplots()
    
    for the_index, (batch, cp) in enumerate(cache_performance_per_batch.items()):
        chunks = np.array_split(cp, number_of_segements)
        accumulative_cache_hit_df = pd.DataFrame(columns=cp.columns)
        hit_ratio_list = []
        percent_number_list = []
        for index, chunk in enumerate(chunks, 1):
            chunk_df = pd.DataFrame(chunk)
            accumulative_cache_hit_df = accumulative_cache_hit_df.append(chunk_df)
            cp_df_hit = accumulative_cache_hit_df.query("cache_hit")
            cp_df_miss = accumulative_cache_hit_df.query("cache_hit == False")
            hit_size = cp_df_hit.shape[0]
            miss_size = cp_df_miss.shape[0]
            hit_ratio = hit_size/(hit_size + miss_size)
            percent_number = (index) * percentage_sizes
            percent_number_list.append(percent_number)
            hit_ratio_list.append(hit_ratio)
        plt.plot(percent_number_list, hit_ratio_list, "-o", color=my_colors[the_index])
        end_hit_ratio = cache_analyse_data['agg_df'].query("batch_id == " + str(batch)).hit_ratio.iloc[0]
        legend.append("Batch :" + str(batch) + " Total Avg Hit Ratio :" + str(end_hit_ratio))
    plt.xlabel('Test Completion Percentage')
    plt.ylabel('Hit Ratio')
    plt.title(title)
    lg = plt.legend(legend, frameon=True, loc='lower right')
    lg.get_frame().set_facecolor('white')
    plt.show()
    fig.savefig(output_dir + file_name + ".pdf", bbox_inches='tight')

### Global Data

In [18]:
batch_df = obtain_df_from_query("select * from batch where end_date is not null")

In [19]:
output_dir = "output_data/"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

## Ehcache Data<a class="anchor" id="ehcache"></a>

### Ehcache LRU 128 MB

In [20]:
ehcache_lru_128_data = crunch_the_data("ehcache", 128, "LRU")
ehcache_lru_128_data["agg_df"]

Average Hit Ratio : 0.0613130973091965


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,1.030268e+09,3,127.076172,127.247070,4357004111650,918,4229,769,0.063372,268,3961
1,1.041366e+09,47,127.022461,127.309570,4306049121045,903,4135,769,0.046433,192,3943
2,1.028875e+09,48,127.176758,127.283203,4324362128599,911,4203,770,0.060195,253,3950
3,1.040019e+09,70,127.246094,127.248047,4319199002360,940,4153,771,0.058993,245,3908
4,1.030743e+09,74,127.182617,126.921875,4340457410974,996,4211,770,0.062930,265,3946
5,1.016581e+09,110,127.181641,127.277344,4292003627503,947,4222,770,0.063477,268,3954
6,1.025874e+09,111,126.881836,127.303711,4294309112202,920,4186,770,0.067129,281,3905
7,1.019053e+09,112,127.285156,127.255859,4346260641583,883,4265,770,0.063775,272,3993
8,1.030396e+09,113,127.243164,127.274414,4289537945444,935,4163,768,0.063896,266,3897
9,1.029211e+09,114,127.186523,127.216797,4334006663309,945,4211,770,0.062930,265,3946


### Ehcache LRU 256 MB

In [21]:
ehcache_lru_256_data = crunch_the_data("ehcache", 256, "LRU")
ehcache_lru_256_data["agg_df"]

Average Hit Ratio : 0.12405093823595922


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,9.554727e+08,1,254.306641,254.633789,8274393451769,1795,8660,1480,0.125173,1084,7576
1,9.499233e+08,50,254.305664,254.675781,8306128912488,1857,8744,1483,0.127287,1113,7631
2,9.605982e+08,54,254.512695,254.792969,8321662253674,1875,8663,1484,0.120628,1045,7618
3,9.538629e+08,73,254.437500,254.527344,8329131077089,1939,8732,1481,0.124943,1091,7641
4,9.599776e+08,76,254.435547,254.615234,8341245253323,1799,8689,1484,0.122224,1062,7627


### Ehcache LRU 512 MB

In [22]:
ehcache_lru_512_data = crunch_the_data("ehcache", 512, "LRU")
ehcache_lru_512_data["agg_df"]

Average Hit Ratio : 0.2515564174600436


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.165716e+08,2,508.772461,509.032227,19398475286350,3777,23756,3525,0.250926,5961,17795
1,8.169449e+08,52,508.711914,508.941406,19288886769051,3733,23611,3529,0.250265,5909,17702
2,8.135592e+08,71,508.840820,509.040039,19339116199300,3892,23771,3523,0.250642,5958,17813
3,8.137362e+08,72,508.854492,509.149414,19348205197601,3769,23777,3529,0.252976,6015,17762
4,8.128092e+08,75,508.929688,509.119141,19403381414974,3749,23872,3530,0.252974,6039,17833


### Ehcache LFU 512 MB

In [23]:
ehcache_lfu_512_data = crunch_the_data("ehcache", 512, "LFU")
ehcache_lfu_512_data["agg_df"]

Average Hit Ratio : 0.2361690150642211


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.289738e+08,83,509.000000,509.013672,19444409213967,3547,23456,3521,0.238660,5598,17858
1,8.331817e+08,84,509.020508,508.725586,19375639433309,3659,23255,3522,0.234960,5464,17791
2,8.375594e+08,85,509.111328,509.134766,19412951701342,3545,23178,3523,0.230520,5343,17835
3,8.300089e+08,86,508.908203,508.932617,19428848239193,3632,23408,3520,0.239875,5615,17793
4,8.369017e+08,87,508.458984,508.840820,19460475945582,3651,23253,3526,0.236830,5507,17746


### Ehcache FIFO 512 MB

In [24]:
ehcache_fifo_512_data = crunch_the_data("ehcache", 512, "FIFO")
ehcache_fifo_512_data["agg_df"]

Average Hit Ratio : 0.2510087991545329


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.170058e+08,88,508.672852,508.908203,19432483326032,3710,23785,3530,0.251629,5985,17800
1,8.157212e+08,89,508.960938,509.195312,19379904402668,3815,23758,3520,0.247411,5878,17880
2,8.119700e+08,90,508.850586,509.115234,19466169035550,3830,23974,3525,0.252440,6052,17922
3,8.179097e+08,91,508.872070,509.145508,19415540929899,3794,23738,3527,0.250105,5937,17801
4,8.138979e+08,92,508.831055,509.144531,19417976801085,3805,23858,3528,0.253458,6047,17811


### Ehcache LRU 1024 MB

In [25]:
ehcache_lru_1024_data = crunch_the_data("ehcache", 1024, "LRU")
ehcache_lru_1024_data["agg_df"]

Average Hit Ratio : 0.5003276082717917


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,5.397724e+08,4,1017.924805,1017.934570,39119461606507,7578,72474,7245,0.506223,36688,35786
1,5.440195e+08,46,1018.046875,1018.131836,38759216120470,7445,71246,7252,0.498863,35542,35704
2,5.443993e+08,49,1017.948242,1018.194336,38877189137294,7500,71413,7248,0.499797,35692,35721
3,5.435268e+08,51,1017.952148,1018.127930,38927387131584,7490,71620,7248,0.499888,35802,35818
4,5.480420e+08,53,1017.843750,1018.013672,38918108224463,7550,71013,7249,0.496867,35284,35729


## Hazelcast Data<a class="anchor" id="hazelcast"></a>

### Hazelcast LRU 128 MB

In [26]:
hazelcast_lru_128_data = crunch_the_data("hazelcast", 128, "LRU")
hazelcast_lru_128_data["agg_df"]

Average Hit Ratio : 0.05842392490730425


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,1.040771e+09,8,132.353516,127.718750,4349381018300,875,4179,772,0.057191,239,3940
1,1.033534e+09,61,129.773438,127.718750,4320171258478,892,4180,772,0.062679,262,3918
2,1.038084e+09,62,127.413086,127.718750,4348535724523,883,4189,772,0.057054,239,3950
3,1.034656e+09,63,130.021484,127.751953,4336243177833,845,4191,772,0.057266,240,3951
4,1.032390e+09,64,136.267578,127.600586,4348427716171,881,4212,772,0.057930,244,3968


### Hazelcast LRU 256 MB

In [27]:
hazelcast_lru_256_data = crunch_the_data("hazelcast", 256, "LRU")
hazelcast_lru_256_data["agg_df"]

Average Hit Ratio : 0.1047117446677313


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,9.880165e+08,7,262.190430,254.733398,8329967124696,1629,8431,1484,0.100581,848,7583
1,9.889869e+08,103,258.569336,254.593750,8347049502146,1618,8440,1483,0.104858,885,7555
2,9.766672e+08,66,256.169922,254.733398,8349527497822,1603,8549,1484,0.107849,922,7627
3,9.753235e+08,67,256.070312,254.852539,8361448364300,1643,8573,1483,0.106031,909,7664
4,9.817675e+08,77,260.781250,254.743164,8335205779862,1613,8490,1483,0.104240,885,7605


### Hazelcast LRU 512 MB

In [28]:
hazelcast_lru_512_data = crunch_the_data("hazelcast", 512, "LRU")
hazelcast_lru_512_data["agg_df"]

Average Hit Ratio : 0.2443262156683808


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.245025e+08,5,515.367188,509.485352,19512676755991,3736,23666,3529,0.247655,5861,17805
1,8.236568e+08,56,510.565430,509.463867,19467128326393,3746,23635,3529,0.243918,5765,17870
2,8.283207e+08,57,512.273438,510.043945,19423291462892,3693,23449,3529,0.239754,5622,17827
3,8.245831e+08,68,516.953125,510.227539,19526952469690,3750,23681,3529,0.243360,5763,17918
4,8.205051e+08,69,514.762695,509.160156,19467303286739,3710,23726,3530,0.246944,5859,17867


### Hazelcast LFU 512 MB

In [29]:
hazelcast_lfu_512_data = crunch_the_data("hazelcast", 512, "LFU")
hazelcast_lfu_512_data["agg_df"]

Average Hit Ratio : 0.24202690666771062


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.279701e+08,78,515.871094,509.937500,19499522865346,3657,23551,3529,0.239183,5633,17918
1,8.263806e+08,79,517.751953,509.267578,19529026272276,3655,23632,3529,0.242214,5724,17908
2,8.283336e+08,80,519.900391,509.965820,19481578449029,3718,23519,3529,0.240912,5666,17853
3,8.247427e+08,81,522.021484,509.495117,19429287616447,3642,23558,3529,0.244036,5749,17809
4,8.242102e+08,82,514.049805,509.160156,19476910894082,3609,23631,3529,0.243790,5761,17870


### Hazelcast LRU 1024 MB

In [30]:
hazelcast_lru_1024_data = crunch_the_data("hazelcast", 1024, "LRU")
hazelcast_lru_1024_data["agg_df"]

Average Hit Ratio : 0.501709512720643


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,5.477232e+08,6,1031.755859,1018.132812,38927236275949,7517,71071,7244,0.497967,35391,35680
1,5.434302e+08,55,1026.896484,1018.143555,38825372671145,7644,71445,7248,0.502037,35868,35577
2,5.436890e+08,58,1027.230469,1018.183594,38906385197538,7581,71560,7245,0.502278,35943,35617
3,5.446865e+08,60,1026.925781,1018.047852,38829614129274,7541,71288,7246,0.500238,35661,35627
4,5.373648e+08,65,1022.010742,1017.898438,38822996654722,7697,72247,7248,0.506028,36559,35688


## Memcached Data<a class="anchor" id="memcached"></a>

### Memcached LRU 128 MB

In [31]:
memcached_lru_128_data = crunch_the_data("memcached", 128, "LRU")
memcached_lru_128_data["agg_df"]

Average Hit Ratio : 0.06037569969959231


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,1.044468e+09,13,112.394531,110.031250,4352298606537,973,4167,692,0.057355,239,3928
1,1.046914e+09,37,112.431641,110.031250,4378194861459,956,4182,692,0.058584,245,3937
2,1.042992e+09,44,112.133789,110.031250,4407684654411,961,4226,692,0.059394,251,3975
3,1.033115e+09,96,112.715820,110.031250,4355613227767,970,4216,692,0.062619,264,3952
4,1.033605e+09,97,113.516602,113.624023,4349409047644,1049,4208,970,0.063926,269,3939


### Memcached LRU 256 MB

In [32]:
memcached_lru_256_data = crunch_the_data("memcached", 256, "LRU")
memcached_lru_256_data["agg_df"]

Average Hit Ratio : 0.10662895989354466


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,9.904896e+08,12,230.811523,224.958984,8418171122703,1702,8499,1368,0.104718,890,7609
1,9.888647e+08,39,229.001953,224.958984,8462704351734,1716,8558,1368,0.104931,898,7660
2,9.814309e+08,42,229.133789,224.958984,8435398772455,1700,8595,1368,0.109366,940,7655
3,9.889570e+08,93,230.411133,224.958984,8385366098645,1702,8479,1368,0.104258,884,7595
4,9.808217e+08,95,230.919922,224.958984,8355620479718,1692,8519,1368,0.109872,936,7583


### Memcached LRU 512 MB

In [33]:
memcached_lru_512_data = crunch_the_data("memcached", 512, "LRU")
memcached_lru_512_data["agg_df"]

Average Hit Ratio : 0.23655077156699228


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.476464e+08,10,460.295898,454.220703,19571308824729,3589,23089,3274,0.235523,5438,17651
1,8.401856e+08,36,458.681641,454.220703,19663703310654,3593,23404,3274,0.239745,5611,17793
2,8.467417e+08,41,458.097656,454.220703,19802749197193,3598,23387,3274,0.238509,5578,17809
3,8.449627e+08,45,459.563477,454.220703,19669041765293,3600,23278,3274,0.232666,5416,17862
4,8.422327e+08,94,459.798828,454.220703,19595385254177,3604,23266,3274,0.236310,5498,17768


### Memcached LRU 1024 MB

In [34]:
memcached_lru_1024_data = crunch_the_data("memcached", 1024, "LRU")
memcached_lru_1024_data["agg_df"]

Average Hit Ratio : 0.44720286094468353


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,6.147017e+08,11,922.816406,921.205078,39785338868101,6740,64723,6593,0.447600,28970,35753
1,6.121219e+08,35,923.733398,921.205078,39492883137224,6740,64518,6593,0.449084,28974,35544
2,6.203688e+08,38,922.737305,921.205078,39732757337859,6739,64047,6593,0.442128,28317,35730
3,6.103817e+08,40,923.976562,921.205078,39939714365104,6739,65434,6593,0.450194,29458,35976
4,6.146303e+08,43,922.592773,921.205078,39719250862958,6739,64623,6593,0.447008,28887,35736


## Redis Data<a class="anchor" id="redis"></a>

### Redis LRU 128 MB

In [35]:
redis_lru_128_data = crunch_the_data("redis", 128, "LRU")
redis_lru_128_data["agg_df"]

Average Hit Ratio : 0.05342096440107451


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,1.062096e+09,14,114.624023,114.500977,4390703781692,875,4134,699,0.051282,212,3922
1,1.055051e+09,19,115.110352,113.568359,4399564586019,883,4170,700,0.059712,249,3921
2,1.058391e+09,20,115.380859,114.900391,4416665100612,869,4173,714,0.051042,213,3960
3,1.060941e+09,27,114.810547,114.640625,4396538498981,866,4144,709,0.055261,229,3915
4,1.059436e+09,30,115.322266,114.819336,4403014073918,836,4156,705,0.049808,207,3949


### Redis LRU 256 MB

In [36]:
redis_lru_256_data = crunch_the_data("redis", 256, "LRU")
redis_lru_256_data["agg_df"]

Average Hit Ratio : 0.10991918027222429


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,9.887766e+08,17,230.288086,231.068359,8473815467128,1679,8570,1340,0.110035,943,7627
1,9.842540e+08,26,230.478516,230.618164,8511829016984,1672,8648,1324,0.114940,994,7654
2,9.922120e+08,29,229.915039,231.077148,8502264905873,1684,8569,1348,0.112032,960,7609
3,9.925420e+08,33,230.184570,230.653320,8578540918384,1698,8643,1341,0.109684,948,7695
4,9.942698e+08,99,229.739258,230.608398,8521886468881,1756,8571,1351,0.102905,882,7689


### Redis LRU 512 MB

In [37]:
redis_lru_512_data = crunch_the_data("redis", 512, "LRU")
redis_lru_512_data["agg_df"]

Average Hit Ratio : 0.22731506812679605


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.610255e+08,16,464.034180,462.880859,19855248877507,3554,23060,3269,0.225369,5197,17863
1,8.570406e+08,22,461.711914,462.874023,19745358247112,3370,23039,3232,0.227354,5238,17801
2,8.650724e+08,24,462.573242,461.243164,19855142781840,3474,22952,3241,0.228738,5250,17702
3,8.633809e+08,31,461.809570,462.390625,19761926291191,3412,22889,3237,0.224081,5129,17760
4,8.554080e+08,32,462.511719,462.870117,19923306672016,3340,23291,3265,0.231033,5381,17910


### Redis LFU 512 MB

In [38]:
redis_lfu_512_data = crunch_the_data("redis", 512, "LFU")
redis_lfu_512_data["agg_df"]

Average Hit Ratio : 0.22621742401006034


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.573325e+08,98,463.356445,462.897461,19842959613962,3373,23145,3194,0.226572,5244,17901
1,8.566102e+08,100,461.819336,463.035156,19858794148571,3425,23183,3223,0.225381,5225,17958
2,8.612698e+08,101,462.536133,462.693359,19937533663421,3386,23149,3169,0.224113,5188,17961
3,8.586268e+08,102,463.213867,462.302734,19829986601736,3436,23095,3172,0.227798,5261,17834
4,8.654456e+08,109,461.894531,462.707031,19881881401633,3456,22973,3209,0.227223,5220,17753


### Redis RANDOM 512 MB

In [ ]:
redis_random_512_data = crunch_the_data("redis", 512, "RANDOM")
redis_random_512_data["agg_df"]

Average Hit Ratio : 0.2264029909066212


,avg_response_time_ns,batch_id,cache_size_mb_end,cache_size_mb_start,elapsed_time_ns,element_count_end,element_count_sent,element_count_start,hit_ratio,hits,misses
0,8.525711e+08,104,463.041992,462.610352,19848706660487,3355,23281,3202,0.229586,5345,17936
1,8.541088e+08,105,462.814453,462.088867,19758099619257,3367,23133,3218,0.230839,5340,17793
2,8.652745e+08,106,464.117188,462.774414,19765466169729,3227,22843,3182,0.221512,5060,17783
3,8.550332e+08,107,463.061523,462.948242,19799149026888,3511,23156,3170,0.226896,5254,17902
4,8.738764e+08,108,462.636719,462.573242,19836121245808,3354,22699,3197,0.223182,5066,17633


### Redis LRU 1024 MB

In [ ]:
redis_lru_1024_data = crunch_the_data("redis", 1024, "LRU")
redis_lru_1024_data["agg_df"]

## Cache Size LRU Comparison<a class="anchor" id="cache_size_comparison"></a>

### 128 MB LRU Comparison

#### Avg Hit Ratio

In [ ]:
plot_avg("128MB LRU Cache Avg Hit Ratio",
                   "hit_ratio",
                    False,
                    ehcache_lru_128_data, 
                    hazelcast_lru_128_data, 
                    memcached_lru_128_data,
                    redis_lru_128_data
                  )

#### Avg Response Time

In [ ]:
plot_avg("128MB LRU Cache Avg Response Time",
                   "avg_response_time_ns",
                    False,
                    ehcache_lru_128_data, 
                    hazelcast_lru_128_data, 
                    memcached_lru_128_data,
                    redis_lru_128_data
                  )

#### Hit Ratio Variation Ehcache 128MB LRU

In [ ]:
plot_hit_ratio_variation("128MB LRU Ehcache Hit Ratio Variation", ehcache_lru_128_data, 1)

#### Hit Ratio Variation Hazelcast 128MB LRU

In [ ]:
plot_hit_ratio_variation("128MB LRU Hazelcast Hit Ratio Variation", hazelcast_lru_128_data, 1)

### 256 MB LRU Comparison

In [ ]:
plot_avg("256MB LRU Cache Avg Hit Ratio", 
                   "hit_ratio",
                    False,
                    ehcache_lru_256_data, 
                    hazelcast_lru_256_data, 
                    memcached_lru_256_data,
                    redis_lru_256_data
                  )

### 512 MB LRU Comparison

In [ ]:
plot_avg("512MB LRU Cache Avg Hit Ratio",
                   "hit_ratio",
                    False,
                    ehcache_lru_512_data, 
                    hazelcast_lru_512_data, 
                    memcached_lru_512_data,
                    redis_lru_512_data
                  )

### 1024 MB LRU Comparison

In [ ]:
plot_avg("1024MB LRU Cache Avg Hit Ratio",
                   "hit_ratio",
                    False,
                    ehcache_lru_1024_data, 
                    hazelcast_lru_1024_data,
                    memcached_lru_1024_data,
                    redis_lru_1024_data
                  )

#### Hit Ratio Variation Ehcache 1024MB LRU

In [ ]:
plot_hit_ratio_variation("1024MB LRU Ehcache Hit Ratio Variation", ehcache_lru_1024_data, 1)

#### Hit Ratio Variation Hazelcast 1024MB LRU

In [ ]:
plot_hit_ratio_variation("1024MB LRU Hazelcast Hit Ratio Variation", hazelcast_lru_1024_data, 1)

### Cache Size

In [ ]:
ehcache_list = [ehcache_lru_1024_data,
                   ehcache_lru_512_data,
                   ehcache_lru_256_data,
                   ehcache_lru_128_data
               ]
hazelcast_list = [hazelcast_lru_1024_data,
                   hazelcast_lru_512_data,
                   hazelcast_lru_256_data,
                   hazelcast_lru_128_data
                 ]
memcached_list = [memcached_lru_1024_data,
                   memcached_lru_512_data,
                   memcached_lru_256_data,
                   memcached_lru_128_data
                 ]
redis_list = [redis_lru_1024_data,
              redis_lru_512_data,
              redis_lru_256_data,
              redis_lru_128_data
             ]

#### Cache Size vs Avg Hit Ratio

In [ ]:
plot_cache_size_vs_avg_line("Avg Hit Ratio vs Cache size",
                                          "hit_ratio",
                                          ehcache_list, 
                                          hazelcast_list, 
                                          memcached_list, 
                                          redis_list)

#### Cache Size vs Avg Response Time

In [ ]:
plot_cache_size_vs_avg_line("Avg Response Time vs Cache size",
                                          "avg_response_time_ns",
                                          ehcache_list, 
                                          hazelcast_list, 
                                          memcached_list, 
                                          redis_list)

## Eviction Policy Comparison<a class="anchor" id="eviction_policy_comparison"></a>

### 512 MB All Policy Comparison

In [ ]:
plot_avg("512MB All Policy Cache Avg Hit Ratio", 
                   "hit_ratio",
                    True,
                    ehcache_lru_512_data,
                    ehcache_fifo_512_data,
                    hazelcast_lru_512_data,
                    hazelcast_lfu_512_data,
                    memcached_lru_512_data,
                    ehcache_lfu_512_data,
                    redis_lru_512_data,
                    redis_random_512_data,
                    redis_lfu_512_data
                  )